In [9]:
%load_ext autoreload
%autoreload 2

In [10]:

from unittest.mock import MagicMock, patch
from src.user.user_management import UserManagement
from src.user.user_model import User
from src.schedule.schedule_model import Schedule
import bcrypt
user_mock = MagicMock()
"""
User have:
 - id
 - username
 - email
 - schedules
 - hashed_password
 - user_preferences
 - get_schedules()
"""

event1 = MagicMock()
event2 = MagicMock()
event3 = MagicMock()
type(event1).title = "event1"
type(event2).title = "event2"
type(event3).title = "event3"

private_schedule = MagicMock()
private_schedule_events = [
    event1, event2, event3
]
# override .elements method

type(user_mock).id = '1'
type(user_mock).username = 'test'
type(user_mock).email = 'test@mail.com'
type(user_mock).schedules = ["private_schedule"]
schedule_dict = {
    "private_schedule": private_schedule
}
salt = bcrypt.gensalt()
type(user_mock).hashed_password = bcrypt.hashpw("test".encode('utf-8'), salt)
type(user_mock).user_preferences = {}
def get_user_schedule_mock(self):
    schedules = self.schedules
    obj_schedules = []
    for schedule in schedules:
        obj_schedules.append(schedule_dict[schedule])
    return obj_schedules
type(user_mock).get_schedules = get_user_schedule_mock
# setting the .events property
type(user_mock).events = private_schedule_events
def get_events_mock(self):
    events = []
    for schedule in self.schedules:
        events += schedule_dict[schedule].get_events()

In [11]:
from src.app.application import Application
import unittest
from src.auth.authentication import AuthenticationModule

from src.app.ui import TkinterUI
# from src.app.states.sign_in_up_state import SignInUp
from src.app.app_states import SignInUp


if __name__ == '__main__':
    db = MagicMock()
    app = Application(db=db)
    # overwrite the authentication of the user \"test\" password \"test\"
    with unittest.mock.patch.object(AuthenticationModule, 'authenticate_user', return_value=True):
        with unittest.mock.patch.object(UserManagement, 'get_user', return_value=user_mock):
            initial_state = SignInUp(app)
            ui = TkinterUI(app)
            app._ui = ui
            app.transition_to(initial_state)
            app.run()
   

Application: Transition to SignInUp
Rendering sign in/up page...
Application: Transition to SignUpState
Rendering sign up page...
Application: Transition to SignInUp
Rendering sign in/up page...
Application: Transition to LoggedOut
Rendering logged out page...
Application: Transition to SignInUp
Rendering sign in/up page...
A janela foi fechada


In [17]:
from src.database.mongo_module import MongoModule
from src.user.user_management import UserManagement
from src.schedule.schedule_management import ScheduleManagement
from src.calendar_elements.element_management import ElementManagement

db = MongoModule(host="localhost", 
                    port=27017,
                    database_name="test_app")
db.connect()    

user_management = UserManagement.get_instance(db)
schedule_management = ScheduleManagement.get_instance(db)
element_management = ElementManagement.get_instance(db)

username = "test3"
email = "aaaa"
password = "test"
user_id = "id_3"


user = user_management.create_user(username, email, password, user_id=user_id)

if user:
    print(f"\033[92mSign up successful! User {username} created.\033[0m")

    schedule_management = ScheduleManagement.get_instance()
    schedule_id = f"{user_id}_schedule"
    schedule_title = "Private schedule"
    schedule_description = f"Private schedule of {username}"
    schedule_permissions = {user_id: "owner"}
    schedule = schedule_management.create_schedule(schedule_id, schedule_title,
                                schedule_description, schedule_permissions, [])

    print(f"\033[92mSchedule created: {schedule}\033[0m")

    user_management.add_schedule_to_user(user_id, schedule_id, "owner")
    print(f"\033[92mSchedule {schedule_id} added to user {user_id}\033[0m")
    user.schedules += [schedule_id]
    user = user_management.get_user(user_id)
    print(f"\033[92mUser schedules: {user.schedules}\033[0m")


Sign up successful! User test3 created.
{'_id': 'id_3', 'username': 'test3', 'email': 'aaaa', 'schedules': [], 'hashed_password': '$2b$12$Iakbou7Uo9PnDX7KgBDneOGwxgOGIT87CM1MYPiqU0Hi73FC9fGYC', 'user_preferences': None}
User(id_3, test3, aaaa,[], {})
User id_3 was updated.
User schedules: ['id_3_schedule']
Schedule created: <src.schedule.schedule_model.Schedule object at 0x000001EEBF112F10>
{'_id': 'id_3', 'username': 'test3', 'email': 'aaaa', 'schedules': ['id_3_schedule'], 'hashed_password': '$2b$12$Iakbou7Uo9PnDX7KgBDneOGwxgOGIT87CM1MYPiqU0Hi73FC9fGYC', 'user_preferences': {}}
User(id_3, test3, aaaa,['id_3_schedule'], {})


DuplicatedIDError: Usuário id_3 já está no schedule id_3_schedule

In [8]:
user_management.get_user(user_id).schedules

{'_id': 'id_2', 'username': 'test2', 'email': 'aaaa', 'schedules': [], 'hashed_password': '$2b$12$KrLOvZ3qlXy5Om3.QuJKe.XyCuDRzvWqMHoP7w.jsJwX.pnGrLD0K', 'user_preferences': None}
User(id_2, test2, aaaa,[], {})


[]

In [15]:
user_management.add_schedule_to_user(user_id, schedule_id, "owner")
user_management.get_user(user_id).schedules

{'_id': 'id_2', 'username': 'test2', 'email': 'aaaa', 'schedules': ['id_2_schedule'], 'hashed_password': '$2b$12$KrLOvZ3qlXy5Om3.QuJKe.XyCuDRzvWqMHoP7w.jsJwX.pnGrLD0K', 'user_preferences': {}, 'password': '$2b$12$KrLOvZ3qlXy5Om3.QuJKe.XyCuDRzvWqMHoP7w.jsJwX.pnGrLD0K'}


TypeError: User.__init__() got an unexpected keyword argument 'password'